In [54]:
import numpy as np

class OLS:
    def __init__(self):
        self.theta = 0

    def fit(self, lambda_, X, Y):
        
        N = X.shape[0]
        p = X.shape[1]
        self.theta = np.linalg.inv((lambda_ * np.eye(p)) + X.T @ X) @ X.T @ Y

    def predict(self, X):
        return X.dot(self.theta)

class active:
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
        self.I = np.ones(self.X.shape[1])
        self.posterior = None

    def get_posterior(self, lambda_, sigma2):
        Sigma = np.linalg.inv((lambda_ * self.I) + sigma2 * (self.X.T @ self.X))
        mu = np.linalg.inv((lambda_ * sigma2 * self.I) + (self.X.T @ self.X)) @ (self.X.T @ self.y)
        self.posterior = [mu,Sigma]
        
    def best_predict(self, D, lambda_, sigma2):
        best_x_val = 0
        best_x_ix = 0
        
        for i,x in enumerate(D):
            sigma2_0 = sigma2 + (x.T @ self.posterior[1] @ x)
            
            if sigma2_0 > best_x_val:
                best_x_val = sigma2_0
                best_x_ix = i
            
        return best_x_val, best_x_ix
    
    def predict_y(self,x, sigma2):
      
        sigma2_0 = sigma2 + (x.T @ (self.posterior[1])@ x)
        mu_0 = x.T.dot(self.posterior[0])
        
        y_0 = np.random.normal(mu_0, sigma2_0)
        return y_0
    
    def update_posterior(self, x_0, y_0, lambda_, sigma2):
        
        Sigma = np.linalg.inv((lambda_*self.I) + 1/sigma2 * (x_0.dot(x_0.T) + (self.X.T.dot(self.X))))
        mu = np.linalg.inv((lambda_ * sigma2 * self.I) + (x_0.dot(x_0.T) + self.X.T.dot(self.X))) * (x_0.dot(y_0) + self.X.T.dot(self.y))
        self.posterior = [mu, Sigma]
        
    def fit(self, D, lambda_, sigma2):
        locations = []
        
        if self.posterior is None:
                self.get_posterior(lambda_, sigma2)
        
        for i in range(10):
            
            x_0,loc = self.best_predict(D, lambda_, sigma2)
            y_0 = self.predict_y(D[loc,:], sigma2)
            self.update_posterior(D[loc,:],y_0, lambda_, sigma2)
            locations.append(loc)
            D = np.delete(D,loc,0)
            
        return locations
                
        
# wrapper functions

def part_1(lambda_, sigma2, X_train, y_train, X_test):
    rgr = OLS()
    rgr.fit(lambda_ ,X_train, y_train)
    w = rgr.theta
    return w

def part_2(lambda_ ,sigma2, X_train, y_train, X_test):
    rgr = active(X_train, y_train)
    return rgr.fit(X_test, lambda_, sigma2)
    

wRR = part_1(lambda_, sigma2, X_train, y_train, X_test)
active = part_2(lambda_, sigma2, X_train, y_train, X_test)

np.savetxt("wRR_" + str(lambda_input) + ".csv", wRR, delimiter="\n") # write output to file
np.savetxt("active_" + str(lambda_input) + "_" + str(int(sigma2_input)) + ".csv", active, delimiter=",") # write output to file



In [51]:
from sklearn.datasets import load_boston

data = load_boston()
X_train = data["data"]
y_train = data["target"]
X_test = X[0:50,:]

lambda_input = 0.9
sigma2_input = 0.2


In [30]:
D.shape

(50, 13)

In [37]:
D = np.delete(D,1,0)
D.shape

(44, 12)

In [28]:
D

array([6.32000e-03, 2.31000e+00, 0.00000e+00, 5.38000e-01, 6.57500e+00,
       6.52000e+01, 4.09000e+00, 1.00000e+00, 2.96000e+02, 1.53000e+01,
       3.96900e+02, 4.98000e+00, 2.73100e-02, 0.00000e+00, 7.07000e+00,
       0.00000e+00, 4.69000e-01, 6.42100e+00, 7.89000e+01, 4.96710e+00,
       2.00000e+00, 2.42000e+02, 1.78000e+01, 3.96900e+02, 9.14000e+00,
       2.72900e-02, 0.00000e+00, 7.07000e+00, 0.00000e+00, 4.69000e-01,
       7.18500e+00, 6.11000e+01, 4.96710e+00, 2.00000e+00, 2.42000e+02,
       1.78000e+01, 3.92830e+02, 4.03000e+00, 3.23700e-02, 0.00000e+00,
       2.18000e+00, 0.00000e+00, 4.58000e-01, 6.99800e+00, 4.58000e+01,
       6.06220e+00, 3.00000e+00, 2.22000e+02, 1.87000e+01, 3.94630e+02,
       2.94000e+00, 6.90500e-02, 0.00000e+00, 2.18000e+00, 0.00000e+00,
       4.58000e-01, 7.14700e+00, 5.42000e+01, 6.06220e+00, 3.00000e+00,
       2.22000e+02, 1.87000e+01, 3.96900e+02, 5.33000e+00, 2.98500e-02,
       0.00000e+00, 2.18000e+00, 0.00000e+00, 4.58000e-01, 6.430